In [1]:
### import all the things ###
import mailbox
import string
import datetime
from dateutil.parser import parse
import random
import email.utils
import email
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.figure
import re
import seaborn as sns
from matplotlib.ticker import *
from collections import Counter
import nltk
from nltk.corpus import stopwords
import lxml.html
from nltk.stem import WordNetLemmatizer, PorterStemmer
import unicodedata
#import gensim

### define all the things ###
def quarters(date):
# a function to print which quarter an email was in

    winter = [1,2,3] #january through march
    spring = [4,5,6] #april through june
    summer = [7,8] #july through september
    fall = [9,10,11,12] #october through december
    if date.month in winter:
        return "WN"
    elif date.month in spring:
        return "SPR"
    elif date.month in summer:
        return "SU"
    elif date.month in fall:
        return "FA"

def d_o_w(date):
#a function that returns the day on which a call was recorded
    
    days_of_week = {0:"Mon", 1:"Tues", 2:"Wed", 3:"Thurs", 4:"Fri", 5:"Sat", 6:"Sun"}
    temp = date.reset_index(drop=True)
    
    #subtract 12 hours as a buffer for late call logs throughout the night, since calls
    #can come in at up to 2 am, but call logs can be submitted at any time in the next day
    
    temp_day = temp["Date"] - pd.Timedelta(hours=12) 
    day = []
    for i in temp_day:
        day.append(days_of_week.get(i.dayofweek))
    return day

""" 
the following two functions are courtesy
of stack overflow user R. Fabbiri

http://stackoverflow.com/questions/7166922/extracting
-the-body-of-an-email-from-mbox-file-decoding-it-to
-plain-text-regard
"""
def getcharsets(msg):
    charsets = set({})
    for c in msg.get_charsets():
        if c is not None:
            charsets.update([c])
    return charsets

def getBody(msg):
    while msg.is_multipart():
        msg=msg.get_payload()[0]
    t=msg.get_payload(decode=True)
    for charset in getcharsets(msg):
        t=t.decode(charset)
    t = lxml.html.fromstring(t.lower()).text_content()
    t = unicodedata.normalize("NFKD", t)
    t = re.sub(r'[\t\n\r]', ' ', t)
    return t

In [2]:
full_mail = mailbox.mbox('../data/All mail Including Spam and Trash -- Most Recent.mbox')
print(type(full_mail))
print(full_mail[0].keys())
print("This mailbox has " + str(len(full_mail)) + " messages.")

<class 'mailbox.mbox'>
['X-GM-THRID', 'X-Gmail-Labels', 'MIME-Version', 'Received', 'Date', 'Delivered-To', 'Message-ID', 'Subject', 'From', 'To', 'Content-Type']
This mailbox has 2007 messages.


In [51]:
n = 200
print("AN EXAMPLE OF THE SUBJECT: " + full_mail[n]["Subject"][:18])
print("\nAN EXAMPLE OF THE RECEPIENTS: " + full_mail[n]["To"]) #was removed during anonymizing of data
print("\nAN EXAMPLE OF THE BODY: " + str(full_mail[n].get_payload()[0])[:360])

AN EXAMPLE OF THE SUBJECT: Midterm Conflicts 

AN EXAMPLE OF THE RECEPIENTS: .

AN EXAMPLE OF THE BODY: Content-Type: multipart/alternative; boundary="001a113d128609f80c056338971a"

--001a113d128609f80c056338971a
Content-Type: text/plain; charset="UTF-8"

Hi ,

I have a midterm conflict, and here are the details:

The exam I will miss: Midterm 1
Why I'm missing it: Physics 137A midterm on the same day, 6-8pm
Two screenshots of 137A syllabus is attached.

Best,


In [52]:
fields = ['Date', 'From', 'To', 'Subject','X-Gmail-Labels', 'X-GM-THRID']
allMessages = []
for message in full_mail.itervalues():
    dmessage = dict(message.items())
    allMessages.append({field:dmessage[field] if field in dmessage.keys() else '' for field in fields})

# process each email and get the body;
# if not, report, and append an empty string
bodies = []
for i in range(len(full_mail)):
    try:
        t = getBody(full_mail[i])
        #t = lxml.html.fromstring(t.lower()).text_content()
        #t = unicodedata.normalize("NFKD", t)
        bodies.append(t)
    except:
        bodies.append("")
# remove pesky replies and signatures
#bodies = [re.sub("on \d\d/\d\d/\d\d.+","",bodies[n]) for n in range(len(bodies))]
#bodies = [re.sub("--.+","",bodies[n]) for n in range(len(bodies))]
#bodies = [re.sub("_+", "",bodies[n]) for n in range(len(bodies))]


mail = pd.DataFrame(allMessages)
mail['Date'] = pd.to_datetime(mail['Date'], utc=True)
#mail["Quarter"]=[quarters(i) for i in mail["Date"]]
mail["Body"] = bodies

In [53]:
print(len(mail))

2007


In [54]:
mail["Body"][3]

"hi ee16a,    i’m so glad you decided to try out gmail. here are a few tips to get you up    and running fast.    --------    gmail        gmail on the go      with gmail for android        and ios    ,    you get real-time notifications so you don't miss important emails.    --------    drive        the best of google    when you sign up for gmail you get a google account        that gives you access to lots of google stuff like 15gb of free storage in    drive    .    check out more tips    .    --------    trust shield        you’re in control    choose what’s right for you. you can review and adjust your privacy and    security settings any time at my account    .    --------    replies to this email aren’t monitored, but if you have a question try the    help center        or post it to the gmail help forum        where my team and other expert users answer questions.      enjoy your new account,    andy        gmail community manager      keep in touch: twitter    ,    facebook  

In [55]:
print(full_mail[0]['Message-ID'])

<CAOh05cst=nEBcX5z8ZC9gViD4Ub9=>


In [56]:
mail.to_csv('../data/mail_tbl -- most recent.csv', index=False, sep=',', encoding='utf-8')